# Chest-prediction

## data cleaning

In [1]:
import numpy as np
import pandas as pd

import os
from pathlib import Path
import subprocess

In [ ]:
PROJECT_NAME = "chest-predictor" #to be adapted depending on the name of the Project Name in your system
NUMBER_OF_IMAGES = '100' # nbr of images to be loaded or 'full' to load the entire dataset (+100k images)

In [2]:
USERNAME = os.environ.get('USER')

In [3]:
LOCAL_DATA_PATH = Path(f"/home/{USERNAME}/code/sachamagier/{PROJECT_NAME}/raw_data/resized_data/images

In [ ]:
def loading_data():
    
    images_data = []
    
    # Define the path to the folder
    folder_path = f'../raw_data/resized_dataset/images/'
    # Get a list of all files in the folder
    file_list = os.listdir(folder_path)

    
    if NUMBER_OF_IMAGES == 'full':
        
        # Loop over the range of folder numbers
        for folder_num in range(1, 13): #13

            # Filter the list to only include image files
            image_files = [f for f in file_list if f.endswith('.png') or f.endswith('.jpg') or f.endswith('.jpeg')]

            # Loop through the image files
            for image_file in image_files:
                # Open the image file
                image = Image.open(folder_path + image_file)
                # Add the image to the list
                images_data.append((image_file, image))
    
    else:

        # Filter the list to only include image files
        image_files = [f for f in file_list if f.endswith('.png') or f.endswith('.jpg') or f.endswith('.jpeg')]
        
        # Loop through the first NUMBER_OF_IMAGES
        for i, image_file in enumerate(image_files):
            
            # Stop the loop after NUMBER_OF_IMAGES iterations
            if i == NUMBER_OF_IMAGES:
                break
            
            # Open the image file
            image = Image.open(folder_path + image_file)
            
            # Add the image to the list
            images_data.append((image_file, image))
    
    return images_data

In [4]:
if LOCAL_DATA_PATH.is_dir():
    print("Load local data...")
    # loading data into data
    
    images_data = loading_data()

else:
    print("Downloading data from Google Drive...")
    
    # downloading data from kaggle and saving it into 'raw_data'
    subprocess.run(["kaggle", "datasets", "download", "-d", "nih-chest-xrays/data", "-p", str(LOCAL_DATA_PATH)])


Traceback (most recent call last):
  File "/Users/arnodebelle/.pyenv/versions/chest_predictor/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/Users/arnodebelle/.pyenv/versions/3.10.6/envs/chest_predictor/lib/python3.10/site-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/Users/arnodebelle/.pyenv/versions/3.10.6/envs/chest_predictor/lib/python3.10/site-packages/kaggle/api/kaggle_api_extended.py", line 164, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /Users/arnodebelle/.kaggle. Or use the environment method.
